# *technische Voraussetzungen*

In [ ]:
# medienfrech'sche IT-Abteilung, 2020-11-21, 2020-11-30
%pip install plotly pillow pandas pip-tools
!pip3 freeze > requirements.txt

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from PIL import Image

# Bild laden

In [ ]:
!curl "..." > a.jpg

img = Image.open('a.jpg').convert('RGB')
img.thumbnail((64, 48), Image.ANTIALIAS)
pix = img.load()
w, h = img.getdata().size

# Bittiefe verringern
farb_bittiefe = lambda v: v // 32 * 32
pixel_bittiefe = lambda rgb: tuple(map(farb_bittiefe, rgb))
for x in range(w):
    for y in range(h):
        pix[x, y] = pixel_bittiefe(pix[x, y])

# img, pix, w, h

# Helligkeitswerte als Höhe interpretieren

In [ ]:
def sub_figure(pix_func, rgb_func):
    return go.Surface(
        z=[[pix_func(x, y) for x in range(w)] for y in range(h)],
        colorscale=[[l/0xff, rgb_func(l)] for l in range(0x100)])

figRGB = make_subplots(rows=1, cols=3, specs=[[{'type': 'surface'} for _ in range(3)]])
figRGB.add_trace(sub_figure(lambda x, y: pix[x, y][0]      , lambda l: f'rgb({l}, 0, 0)'), row=1, col=1)
figRGB.add_trace(sub_figure(lambda x, y: pix[x, y][1]      , lambda l: f'rgb(0, {l}, 0)'), row=1, col=2)
figRGB.add_trace(sub_figure(lambda x, y: pix[x, y][2]      , lambda l: f'rgb(0, 0, {l})'), row=1, col=3)

fig = go.Figure(data=sub_figure(lambda x, y: sum(pix[x, y]) / 3, lambda l: f'rgb({l}, {l}, {l})'))

fig.show()
figRGB.show()

# Helligkeitswerte positionsambivalent aufzeichnen

In [ ]:
grauwert = lambda x, y: sum(pix[x, y]) / 3

histogram = [len([None for x in range(w) for y in range(h) if int(grauwert(x, y)) == k])/w/h for k in range(256)]
fig = go.Figure([go.Bar(x=list(range(256)), y=histogram)])
fig.show()

# Experimentell

In [ ]:
"""
import plotly.express as px
px.imshow([[pixel_to_plot(x, y) for x in range(w)] for y in range(h)]).show()

def erhöhe_kontrast(min_helligkeit, max_helligkeit):
    for y in range(h):
        for x in range(h):
            r, g, b = pix[x, y]
            k = (r + g + b) / 3
            0<=k<=255
            k2 = (k-min_helligkeit) * 255 / (max_helligkeit-min_helligkeit)
            k2 = max(0, min(255, int(k2)))
            pix[x, y] = (k2, k2, k2)
            
erhöhe_kontrast(50, 200)

px.imshow([[pixel_to_plot(x, y) for x in range(w)] for y in range(h)]).show()

histogram = [len([None for x in range(w) for y in range(h) if int(pixel_to_plot(x, y)) == k])/w/h for k in range(256)]
fig = go.Figure([go.Bar(x=list(range(256)), y=histogram)])
fig.show()
"""